In [1]:
# # !pip install --upgrade langchain
# # !pip install --upgrade langchain-groq
# # !pip install -U langchain-community
# # !pip install python-docx pypandoc
# # !pip install python-docx docx2pdf

# # !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install cryptography

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
import pypandoc
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq

api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [3]:
from cryptography.fernet import Fernet
import base64
from dotenv import load_dotenv
import os
# Load environment variables from .env
load_dotenv()
with open("D:\\DEV\\WebdevFolder\\key.txt", 'r') as file:
    k = file.read().strip()[2:-1]
    #k = base64.urlsafe_b64decode(k.encode())


# Decode and decrypt the API key
encoded_encrypted_api_key = os.getenv("ENCRYPTED_API_KEY")
encrypted_api_key = base64.b64decode(encoded_encrypted_api_key)
decipher_suite = Fernet(k)


In [4]:
from QuoteUtils import initialize_web_search_agent, initialize_quote_bot

cc_out = initialize_quote_bot(client,llm)

Hello! I'm excited to help you with your renovation project quote. Let's get started!

Can you please provide the address and location of the property you're planning to renovate? This will help me better understand the project scope and requirements.
I see you've already been asked about the address and location of the property! Since I don't see an answer to that question, let me reiterate it for clarity.

Can you please provide the address and location of the property you're planning to renovate? This will help me better understand the project scope and requirements.
It looks like we've already got the address and location of the property, which is 3210 Greenstone Way, Herndon, VA 20171. 

Let's move on to the next question. What type of property is this? Is it a residential or commercial property?
hehehe


d:\DEV\WebdevFolder\RealEstateAI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




> Entering new AgentExecutor chain...
Action: ```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "Renovation project quote for 3210 Greenstone Way, Herndon, VA 20171 with [insert property type] and the following materials: [list materials and quantities above]"
  }
}
```[{'url': 'https://www.redfin.com/VA/Herndon/3215-Greenstone-Way-20171/home/9423762', 'content': "What's the housing market like in Herndon? 4 beds, 4 baths, 2267 sq. ft. house located at 3215 Greenstone Way, Oak Hill, VA 20171 sold for $545,000 on Feb 15, 2011. MLS# 1005228955."}, {'url': 'https://www.zillow.com/homedetails/3210-Greenstone-Way-Herndon-VA-20171/51760858_zpid/', 'content': '3210 Greenstone Way, Herndon VA, is a Single Family home that contains 2200 sq ft and was built in 1980.It contains 6 bedrooms and 3 bathrooms.This home last sold for $687,000 in August 2020. The Zestimate for this Single Family is $822,600, which has decreased by $8,081 in the last 30 days.The Rent Zest

In [5]:
from pprint import pprint
print(cc_out)

RENNOVATION QUOTE DOCUMENT

Project Address: 3210 Greenstone Way, Herndon, VA 20171

**Project Overview:**
- Project Description: Renovation project at 3210 Greenstone Way, Herndon, VA 20171
- Length of Time for Project: Not specified

**Cost Breakdown:**

1. **Labor Costs:**
- Total Labor Cost: $20,000

2. **Material Costs:**
- Total Material Cost: $24,200
- Itemized Costs:
    + 1,500 sqft of 1/2" drywall: $1,800
    + 300 sqft of 5/8" drywall: $450
    + 1,000 sqft of R-13 fiberglass batt insulation: $700
    + 500 sqft of R-19 fiberglass batt insulation: $350
    + 1,000 sqft of 3/4" thick x 5" wide oak hardwood flooring: $4,500
    + 200 sqft of ceramic tile: $1,200
    + 100 sqft of carpeting: $500
    + 5 gallons of premium latex paint: $250
    + 2 gallons of semi-gloss paint: $150
    + 1,000 linear feet of 3-1/2" MDF baseboard molding: $700
    + 500 linear feet of 2-1/4" MDF door and window casing: $350
    + 1 pedestal sink with single-hole faucet: $700
    + 1 toilet with 

In [6]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re

llm.groq_api_key = random.choice(api_keys)

def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
    # Check if the text should be a bullet point
    if text.startswith("- ") or text.startswith("+ "):
        paragraph = doc.add_paragraph(style='ListBullet')
        text = text[2:]  # Remove the bullet point indicator
    else:
        paragraph = doc.add_paragraph()
    
    run = paragraph.add_run(text)
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    run.italic = italic
    if color:
        run.font.color.rgb = RGBColor(*color)
    
    if alignment == 'center':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


def write_file(title, sections, append=False):
    documents_path = os.path.join(os.getcwd(), 'documents')
    os.makedirs(documents_path, exist_ok=True)
    docx_path = os.path.join(documents_path, f'{title}.docx')
    pdf_path = os.path.join(documents_path, f'{title}.pdf')
    
    if append and os.path.exists(docx_path):
        doc = Document(docx_path)
    else:
        doc = Document()

    for section in sections:
        add_section(
            doc,
            section["content"],
            font_name=section.get("font_name", "Arial"),
            font_size=section.get("font_size", 12),
            bold=section.get("bold", False),
            italic=section.get("italic", False),
            alignment=section.get("alignment", "left"),
            color=section.get("color", None)
        )
    
    doc.save(docx_path)
    convert(docx_path, pdf_path)

    return f"Document written and converted to pdf. No further action needed."


class WriteFileTool(BaseTool):
    name = "write_file"
    description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file.\
        If you encounter errors, use this tool ONLY and input everything properly and try again.\
            IMPORTANT! - DO NOT CALL TOOL MULTIPLE TIMES IF FIRST TIME WORKED ALREADY!"
    
    def _run(self, title, sections, append=False):
        self.process_sections_in_batches(title, sections, append)

    def process_sections_in_batches(self, title, sections, append, batch_size=1):
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i+batch_size]
            result = write_file(title, batch, append=(i > 0))
        return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE!")


# def parse_formatted_response(response):
#     title_match = re.search(r'"title":\s*"([^"]+)"', response)
#     title = title_match.group(1) if title_match else "Untitled Document"
    
#     sections_matches = re.findall(r'\{([^}]+)\}', response)
#     sections = []
#     for match in sections_matches:
#         section = {}
#         content_match = re.search(r'"content":\s*"([^"]+)"', match)
#         font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
#         font_size_match = re.search(r'"font_size":\s*(\d+)', match)
#         bold_match = re.search(r'"bold":\s*(true|false)', match)
#         italic_match = re.search(r'"italic":\s*(true|false)', match)
#         alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
#         color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
#         section["content"] = content_match.group(1) if content_match else ""
#         section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
#         section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
#         section["bold"] = bold_match.group(1) == "true" if bold_match else False
#         section["italic"] = italic_match.group(1) == "true" if italic_match else False
#         section["alignment"] = alignment_match.group(1) if alignment_match else "left"
#         if color_match:
#             section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
#         sections.append(section)
    
#     append_match = re.search(r'"append":\s*(true|false)', response)
#     append = append_match.group(1) == "true" if append_match else False
    
#     return title, sections, append

import re

def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        if content_match:
            content = content_match.group(1)
            # Replace double quotes indicating inches with the word "inch"
            content = re.sub(r'(?<=\d)"', ' inch', content)
            section["content"] = content
        else:
            section["content"] = ""
        
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append

# Example usage
response = """
{
  "title": "The Great Renovation",
  "sections": [
    {
      "content": "500 sqft of 1/2\\" thick drywall for walls and ceilings: $1,500",
      "font_name": "Times New Roman",
      "font_size": 18,
      "bold": false,
      "italic": false,
      "alignment": "left",
      "color": [0, 0, 0]
    },
    {
      "content": "- 1,000 sqft of 3.5\\" wide engineered hardwood flooring in oak: $8,000",
      "font_name": "Times New Roman",
      "font_size": 12,
      "bold": false,
      "italic": false,
      "alignment": "left",
      "color": [0, 0, 0]
    }
  ],
  "append": false
}
"""

title, sections, append = parse_formatted_response(response)
print("Title:", title)
print("Sections:", sections)
print("Append:", append)


chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"You are a professional formatter. BE ELABORATE. REPLACE all quotations representing inches to the word 'inch'"
        },
        {
            "role":"user",
            "content":"please synthesize ALL of this stuff into sentence format. Start the paragraph with '@' character. put the itemized costs in parentheses beside the item: " + cc_out,
        }
    ],
    model = "llama3-70b-8192",
)

cc_out2 = (chat_completion.choices[0].message.content)
idx = cc_out2.find('@')
cc_out2 = cc_out2[idx+1:].strip()
print('ccount2 old\n',cc_out2)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a detailed and professional business assistant. Your task is to generate comprehensive and detailed formatted text. 
            Ensure all items, costs, and specifications are included without summarizing or truncating any content.
            Your response must be well-formed and include all details EVEN THE EXPLICIT ADDRESSS!!. Do NOTTTTTTT use phrases like "and so on for the other materials." List every item explicitly.
            You must always output in a structured format as follows:
            
                        You must always output in a structured format as follows:
                        {
                            "title": "The Great Renovation",
                            "sections": [
                                {
                                    "content": "<specific text content>",
                                    "font_name": "<font family>",
                                    "font_size": <integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                },
                                {
                                    "content": "<another specific text content>",
                                    "font_name": "<another font family>",
                                    "font_size": <another integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                }
                                // ... more sections as needed
                            ],
                            "append": <true or false> (this indicates appending to document or not)
                        }
                        Ensure the output is well-formed and valid."""
        },
        {
            "role": "user",
            "content": "TITLE OF DOCUMENT IS 'The Great Renovation'. Include titles, headings, format in a nice readable and coherent way, NOT a big block of text.  \
                Specify the formatting for this text. Please make it a \
                    PROFESSIONALLY formatted text WITH black blue colors and times new roman font PLEASE.: " + cc_out2
        }
    ],
    model="llama3-70b-8192",
)

cc_out2 = chat_completion.choices[0].message.content

print('CCOUT2\n',cc_out2)

# title, sections, append = parse_formatted_response(cc_out2)

# write_file_tool._run(title, sections, append)


Title: The Great Renovation
Sections: [{'content': '500 sqft of 1/2\\', 'font_name': 'Times New Roman', 'font_size': 18, 'bold': False, 'italic': False, 'alignment': 'left', 'color': (0, 0, 0)}, {'content': '- 1,000 sqft of 3.5\\', 'font_name': 'Times New Roman', 'font_size': 12, 'bold': False, 'italic': False, 'alignment': 'left', 'color': (0, 0, 0)}]
Append: False
ccount2 old
 Here is the comprehensive renovation quote document for the project at 3210 Greenstone Way, Herndon, VA 20171, which encompasses a renovation project with an unspecified timeline, featuring a total labor cost of $20,000 and a total material cost of $24,200, comprising (among other items) 1,500 sqft of 1/2 inch drywall ($1,800), 300 sqft of 5/8 inch drywall ($450), 1,000 sqft of R-13 fiberglass batt insulation ($700), 500 sqft of R-19 fiberglass batt insulation ($350), 1,000 sqft of 3/4 inch thick x 5 inch wide oak hardwood flooring ($4,500), 200 sqft of ceramic tile ($1,200), 100 sqft of carpeting ($500), 5 gal

In [7]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient
import google.auth.exceptions

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive.")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        credentials_path = "D:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

        # Authenticate and create the service
        try:
            credentials = service_account.Credentials.from_service_account_file(credentials_path)
            service = build('drive', 'v3', credentials=credentials)
            print("Successfully authenticated and created the service.")
        except google.auth.exceptions.GoogleAuthError as e:
            return f"Authentication error: {e}"

        # File details
        if rename is not None:
            file_metadata = {'name': rename}
        else:
            file_metadata = {'name': os.path.basename(file_path)}

        if not os.path.exists(file_path):
            return f"File not found: {file_path}"
        else:
            media = MediaFileUpload(file_path, mimetype='application/pdf')
            try:
                # Upload file
                file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
                file_id = file.get('id')
                print(f"File ID: {file_id}")

                # Verify the upload by fetching the file details
                file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
                print("File details:")
                print(file_info)

                # Sharing settings
                permission = {
                    'type': 'user',
                    'role': 'writer',  # Set to 'reader' if read-only access is needed
                    'emailAddress': user_email
                }

                # Grant permission to the specific user
                try:
                    service.permissions().create(
                        fileId=file_id,
                        body=permission,
                        fields='id'
                    ).execute()
                    
                    print(f"Granted {permission['role']} access to {user_email}")
                    
                    

                except googleapiclient.errors.HttpError as e:
                    print(f"Error setting permission: {e}")
                    return f"Error setting permission: {e}"

                return f"File uploaded and permissions set successfully. File ID: {file_id}"

            except Exception as e:
                return f"Error during file upload or conversion: {e}"

    

    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

# Example usage
# tool = GoogleDriveUploadTool()
# response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# print(response)


In [8]:
zapier = ZapierNLAWrapper(zapier_nla_api_key=decipher_suite.decrypt(encrypted_api_key).decode())
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
import json
from langchain.agents import Tool
my_tools = []
for tool in toolkit.tools:
    my_tools.append(tool)


my_tools.append(
    
    WriteFileTool(),
    
)

my_tools.append(
    
    GoogleDriveUploadTool()
)
llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

d:\DEV\WebdevFolder\RealEstateAI\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
for tool in my_tools:
    print(tool.name)
    print(tool.description)
    print("\n\n")

Google Drive: Upload File
A wrapper around Zapier NLA actions. The input to this tool is a natural language instruction, for example "get the latest email from my bank" or "send a slack message to the #general channel". Each tool will have params associated with it that are specified as a list. You MUST take into account the params when creating the instruction. For example, if the params are ['Message_Text', 'Channel'], your instruction should be something like 'send a slack message to the #general channel with the text hello world'. Another example: if the params are ['Calendar', 'Search_Term'], your instruction should be something like 'find the meeting in my personal calendar at 3pm'. Do not make up params, they will be explicitly specified in the tool description. If you do not have enough information to fill in the params, just say 'not enough information provided in the instruction, missing <param>'. If you get a none or null response, STOP EXECUTION, do not try to another tool!

In [10]:
title, sections, append = parse_formatted_response(cc_out2)
llm.groq_api_key = random.choice(api_keys)
llm.temperature = 0.2
for i in range(5):
    try:
        result = agent.invoke({"input": f"use ONLY write file tool. INPUT EVERYTHING PROPERLY! --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
    except:
        i += 1
        continue
    if(i >= 5):
        break

print(result)



> Entering new AgentExecutor chain...
Here is a new attempt at the Completion:

Action:
```
{
  "action": "write_file",
  "action_input": "The Great Renovation"
}
```

Please let me know if this satisfies the constraints.

> Entering new AgentExecutor chain...
I apologize for the mistake. Here is a new attempt at the Completion:

Action:
```
{
  "action": "write_file",
  "action_input": "The Great Renovation"
}
```

Please let me know if this meets the requirements.

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 12, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "COMPREHENSIVE RENOVATION QUOTE", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Cost: $

100%|██████████| 1/1 [00:01<00:00,  1.02s/it]



Observation: None
Thought:Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "3210 Greenstone Way, Herndon, VA 20171", "font_name": "Times New Roman", "font_size": 12, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "COMPREHENSIVE RENOVATION QUOTE", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Labor Cost: $20,000", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Material Cost: $24,200", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "MATERIALS:", "font_name": "Times New Roman", "font_size": 12, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "\u2022 1,500 sqft of 1/2 inch d

In [11]:
import os
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

def authenticate_google_sheets():
    # Load credentials from the service account file
    creds = service_account.Credentials.from_service_account_file(
        "D:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json", scopes=SCOPES)
    return creds

def upload_to_google_sheets(spreadsheet_id, range_name, values):
    creds = authenticate_google_sheets()
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    body = {
        'values': values
    }
    result = sheet.values().update(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption="RAW", body=body).execute()
    print(f"{result.get('updatedCells')} cells updated.")

def append_row_to_google_sheets(spreadsheet_id, range_name, values):
    """
    Appends a row to the specified range in the Google Sheets spreadsheet.
    
    :param spreadsheet_id: The ID of the spreadsheet to update.
    :param range_name: The range to append to, typically a sheet name (e.g., 'Sheet1').
    :param values: The data to append, as a list of lists (e.g., [['A1', 'B1', 'C1']]).
    """
    creds = authenticate_google_sheets()
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    body = {
        'values': values
    }
    result = sheet.values().append(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption="RAW", body=body,
        insertDataOption="INSERT_ROWS"
    ).execute()
    print(f"{result.get('updates').get('updatedCells')} cells appended.")


In [12]:
def extract_file_id(output):
    match = re.search(r"File ID: ([^'}]+)", output)
    if match:
        file_id = match.group(1).strip()
        return file_id
    else:
        raise ValueError("File ID not found in the output string")

In [14]:
file_path, user_email = "./documents/The Great Renovation.pdf", "thegreatchrisjohn@gmail.com"

import random
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials


file_path = "./documents/The Great Renovation.pdf"
user_email = "sriramnallani35@gmail.com"


llm.groq_api_key = random.choice(api_keys)
llm.temperature = 0.2
result = agent.invoke({"input": f"please use google drive upload tool, rename the file to 'Quote' --> FILE_PATH: {file_path} USER_EMAIL: {user_email}"})
print(result)

fileID = extract_file_id(str(result))

link = f"https://drive.google.com/file/d/{fileID}/view"
print(link)
# Example usage
if __name__ == '__main__':
    SPREADSHEET_ID = '1TSFyiTwctC1tABr2RQouBzLRMCG4RZ7lXdTVi-I58Mo'  # Replace with your actual spreadsheet ID
    RANGE_NAME = 'Sheet1'  # The sheet name to append to
    names = ['ayya babooi', 'Muthukumaran Ramakrishnan Sundararajan', 'Sundar Pichai', 'Satya Nadella', 'Elon Musk', 'Ooga booga']
    nos = ['+1 (234) 567-8901','+44 7700 900234','+61 412 345 678','+91 98765 43210','+49 1512 3456789','+33 612 345 678']
    VALUES = [
        [random.choice(names), random.choice(nos), link],
    ]

    append_row_to_google_sheets(SPREADSHEET_ID, RANGE_NAME, VALUES)

llm.temperature = 0.7



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "GoogleDriveUploadTool",
  "action_input": {
    "file_path": "./documents/The Great Renovation.pdf",
    "user_email": "sriramnallani35@gmail.com",
    "rename": "Quote"
  }
}
```
Successfully authenticated and created the service.
File ID: 1ExFjVDQoB-bnurlTqmlSKuATYKHPUbMr
File details:
{'id': '1ExFjVDQoB-bnurlTqmlSKuATYKHPUbMr', 'name': 'Quote', 'parents': ['0AJW9tsOpMsePUk9PVA']}
Granted writer access to sriramnallani35@gmail.com

Observation: File uploaded and permissions set successfully. File ID: 1ExFjVDQoB-bnurlTqmlSKuATYKHPUbMr
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "File uploaded and permissions set successfully. File ID: 1ExFjVDQoB-bnurlTqmlSKuATYKHPUbMr"
}
```


> Finished chain.
{'input': "please use google drive upload tool, rename the file to 'Quote' --> FILE_PATH: ./documents/The Great Renovation.pdf USER_EMAIL: sriramnallani35@gmail.com", 'output': 'File uploaded and permissi